In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [2]:
dfData = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv')
dfTrain = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/labels_training_set.csv')
dfTest = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')
#dfSample = pd.read_csv('trocafone_kaggle_submit_sample_all_0.csv')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Crea un df con todos los usuarios
dfTest['label'] = 0
dfUsers = pd.concat([dfTrain,dfTest])
dfTest = dfTest.drop('label', axis=1)

In [4]:
eventos2=dfData.copy()
modelos_popularesdf=eventos2[eventos2['event']=='conversion'].groupby(['model'])['model'].count().rename('cantidad').sort_values(ascending=False).reset_index()

total_ventas=modelos_popularesdf['cantidad'].sum()
modelos_popularesdf['peso modelo']=modelos_popularesdf['cantidad']/total_ventas*1000
modelos_popularesdf = modelos_popularesdf.drop('cantidad', axis=1)
modelos_popularesdf.head()

,model,peso modelo
0,Samsung Galaxy J5,73.191369
1,iPhone 5s,69.947821
2,iPhone 6,56.127486
3,iPhone 6S,43.294317
4,Motorola Moto G2 3G Dual,38.358483


In [5]:
colores_popularesdf=eventos2[eventos2['event']=='conversion'].groupby(['color'])['color'].count().rename('cantidad').sort_values(ascending=False).reset_index()

total_colores=colores_popularesdf['cantidad'].sum()
colores_popularesdf['peso color']=colores_popularesdf['cantidad']/total_colores*10
colores_popularesdf = colores_popularesdf.drop('cantidad', axis=1)
colores_popularesdf.head()

,color,peso color
0,Preto,3.268932
1,Dourado,2.424200
2,Branco,1.280496
3,Cinza espacial,0.702299
4,Prateado,0.434353


In [6]:
dfData=dfData.merge(colores_popularesdf,how='left',on='color')

In [7]:
dfData=dfData.merge(modelos_popularesdf,how='left',on='model')

dfData.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,peso color,peso modelo
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.424200,30.884219
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.702299,69.947821
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.434353,43.294317
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018333,12.269073
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.280496,0.987167


In [8]:
#Le asigna a cada usuario su cantidad de eventos dandole mas valor si son mas recientes
def pesoTiempo(time):
    mes = int(time[6])
    dia = int(time[8:10])
    if mes < 5:
        return 1
    return 10 + dia * 0.7

In [9]:
dfPE = dfData[['person', 'event', 'timestamp','peso modelo','peso color']]
dfPE.head()

,person,event,timestamp,peso modelo,peso color
0,4886f805,viewed product,2018-05-18 00:11:59,30.884219,2.424200
1,ad93850f,viewed product,2018-05-18 00:11:27,69.947821,0.702299
2,0297fc1e,viewed product,2018-05-18 00:11:16,43.294317,0.434353
3,2d681dd8,viewed product,2018-05-18 00:11:14,12.269073,0.018333
4,cccea85e,viewed product,2018-05-18 00:11:09,0.987167,1.280496


In [10]:
dfPE['puntajeTiempo'] = dfPE['timestamp'].apply(pesoTiempo)
dfPE = pd.get_dummies(dfPE, columns=['event'])
dfPE = dfPE.drop(['event_generic listing', 'event_lead'], axis=1)
columnasEventos = dfPE.columns.values[3:]
for eventoColumna in columnasEventos:
    dfPE[eventoColumna] *= dfPE['puntajeTiempo']
dfPE = dfPE.drop(['timestamp', 'puntajeTiempo'], axis=1).groupby('person').sum()
dfUsers = dfUsers.sort_values('person').merge(dfPE, on='person')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
dfPE.head()

,peso modelo,peso color,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
person,,,,,,,,,,,
0008ed71,36.525173,140.059935,0.00,0.00,1438.83,0.00,0.00,0.00,0.0,0.00,959.22
00091926,7470.878578,12401.656748,9740.26,10877.88,1371.24,0.00,0.00,0.00,0.0,181314.25,16503.51
00091a7a,109.857566,3.433930,1.00,5.00,0.00,0.00,0.00,0.00,0.0,3.00,1.00
000ba417,1953.744183,8120.339726,479.61,13966.31,3670.55,795.24,479.61,0.00,0.0,104364.78,3747.55
000c79fe,49.076294,11.964462,918.09,0.00,918.09,0.00,918.09,8262.81,0.0,2754.27,918.09


In [12]:
#Le asigna a cada usuario la region en la que mas eventos tiene
regionesMasEventos = dfData['region'].value_counts().head(9).index.tolist()
def filtrarRegiones(region):
    if pd.isnull(region):
        return 'null'
    if region not in regionesMasEventos:
        return 'Other'
    return region
    
dfPE = dfData[['person', 'region']]
dfPE['regionAcotada'] = dfPE['region'].apply(filtrarRegiones)

lDictRegiones = []
temp = dfPE.drop('region', axis=1).groupby('person')
for name, group in temp:
    regiones = {'null' : 1}
    for row in group.itertuples():
        region = row.regionAcotada
        if region == 'null':
            continue
        if region in regiones:
            regiones[region] += 1
        else:
            regiones[region] = 1
        rMax = max(regiones, key=regiones.get)
    lDictRegiones.append({'person' : name, 'region' : rMax})

dfRegiones = pd.DataFrame(lDictRegiones)
dfRegiones = pd.get_dummies(dfRegiones, columns=['region'])
dfUsers = dfUsers.merge(dfRegiones, on='person')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:
#Casteo dfUsers a int32
personas = dfUsers['person']
dfUsers.drop('person', axis=1, inplace=True)
dfUsers = dfUsers.astype('int32')
dfUsers['person'] = personas

In [14]:
#Agrego columna con ultima visita al sitio
temp = dfData.groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, on='person')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_visit'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [15]:
#Agrego columna con ultima compra
temp = dfData.loc[dfData['event'].isin(['conversion','lead','checkout'])]\
                                 .groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, how='left', on='person')
dfUsers = dfUsers.fillna('000000000000000000')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_conversion'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [16]:
#Columna con relacion entre el uso del celular y de la computadora
def filtrarDispositivo(deviceType):
    if deviceType == 'Computer':
        return 1
    elif deviceType == 'Smartphone':
        return 0
    else:
        return 2
    
dfPE = dfData[['person', 'device_type']]
dfPE['dispositivo'] = dfPE['device_type'].apply(filtrarDispositivo)
dfPE = dfPE.loc[dfPE['dispositivo'] != 2].drop('device_type', axis=1).groupby('person').mean()
dfUsers = dfUsers.merge(dfPE, on='person', how='left').fillna(0)

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [17]:
#Feature storage
dfT = dfData.loc[dfData['event'] == 'viewed product'][['person', 'storage']].dropna()
dfT = dfT.loc[dfT['storage'] != '256GB']
dfT = dfT.loc[dfT['storage'] != '4GB']
dfT = dfT.loc[dfT['storage'] != '512MB']
dfT = dfT.loc[dfT['storage'] != '8GB']
dfT = dfT.loc[dfT['storage'] != '128GB']
dfQChannels = dfT.groupby('person')['storage'].count().to_frame()
dfPE = pd.get_dummies(dfT, columns=['storage']).groupby('person').sum()
columnasCanales = dfPE.columns.values
dfPE = dfPE.merge(dfQChannels, on='person')
for canalColumna in columnasCanales:
    dfPE[canalColumna] /= dfPE['storage']
dfPE = dfPE.drop('storage', axis=1)
dfUsers = dfUsers.merge(dfPE, on='person', how='left').fillna(0)

In [18]:
dfUsers.head()

,label,peso modelo,peso color,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_search engine hit,event_searched products,event_staticpage,...,region_Sao Paulo,region_Unknown,region_null,person,last_visit,last_conversion,dispositivo,storage_16GB,storage_32GB,storage_64GB
0,0,36,140,0,0,1438,0,0,0,0,...,0,1,0,0008ed71,167,167,1.0,0.000000,0.000000,0.000000
1,0,7470,12401,9740,10877,1371,0,0,0,0,...,0,0,0,00091926,181,176,1.0,0.331210,0.420382,0.248408
2,0,109,3,1,5,0,0,0,0,0,...,0,0,1,00091a7a,116,0,0.0,0.500000,0.000000,0.500000
3,0,1953,8120,479,13966,3670,795,479,0,0,...,1,0,0,000ba417,176,176,1.0,0.837209,0.155039,0.007752
4,0,49,11,918,0,918,0,918,8262,0,...,0,0,1,000c79fe,179,179,0.0,0.000000,0.000000,0.000000


In [19]:
df_y = dfTrain['label']
df_X = dfTrain.drop('label', axis=1).merge(dfUsers, on='person').drop(['person','label'], axis=1)
df_X.head()

,peso modelo,peso color,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_search engine hit,event_searched products,event_staticpage,event_viewed product,...,region_Rio de Janeiro,region_Sao Paulo,region_Unknown,region_null,last_visit,last_conversion,dispositivo,storage_16GB,storage_32GB,storage_64GB
0,535,1134,4272,1971,681,681,645,0,718,14946,...,0,0,0,0,181,173,0.0,0.500000,0.500000,0.0
1,0,0,876,0,0,0,0,0,0,0,...,0,0,0,1,178,0,0.0,0.000000,0.000000,0.0
2,1254,728,137,1189,918,0,4,1840,0,12082,...,0,1,0,0,179,179,0.0,0.366667,0.333333,0.3
3,252,1355,0,3781,0,0,510,0,0,18267,...,0,1,0,0,177,0,0.0,0.652174,0.347826,0.0
4,190,597,2553,0,1021,0,0,0,0,4628,...,0,0,0,1,169,168,1.0,1.000000,0.000000,0.0


# Gradient Boosting

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score


GBC = GradientBoostingClassifier(random_state=23, n_estimators=50, min_samples_split=50)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8734441882509751

# XGBoost

In [21]:
#Pruebo con XGBoost
XGBC = xgb.XGBClassifier(learning_rate =0.075, n_estimators=95, max_depth=4, min_child_weight=6, 
                         gamma=0.3, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',
                         scale_pos_weight=0.8, seed = 15)

scores = cross_val_score(XGBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.876589846272776

In [22]:
# hago una randomized searh para encontarr los parametros optimos de XGBoost
params = {
          'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(XGBC, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(df_X,df_y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(df_X, df_y)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=0.8826682829356316, total=   7.0s
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=0.8895028312382723, total=   7.0s
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=0.8665401880849981, total=   7.1s
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma

[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  1.0min finished



 Time taken: 0 hours 1 minutes and 9.66 seconds.


In [23]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([ 6.92134809, 10.09771428, 10.80417113,  8.6921145 ,  9.04437103]), 'std_fit_time': array([0.10317735, 1.09661932, 1.2029635 , 0.64686224, 1.60251967]), 'mean_score_time': array([0.06124172, 0.10745153, 0.11544571, 0.06736875, 0.06528816]), 'std_score_time': array([0.00653509, 0.04818222, 0.0233865 , 0.00628889, 0.03108346]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
         

In [24]:
# XGBoost con nuevos parametros
XGBC = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1.0, gamma=1, learning_rate=0.075,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=95, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=0.8, seed=15, silent=True,
       subsample=0.8)

scores = cross_val_score(XGBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8755765302871328

In [31]:
XGB2 = xgb.XGBClassifier(learning_rate =0.075, n_estimators=95, max_depth=4, min_child_weight=6, 
                         gamma=0.3, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',
                         scale_pos_weight=0.8, seed = 15)
scores = cross_val_score(XGB2 , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.876589846272776

In [25]:
from sklearn.ensemble import RandomForestClassifier

rdf = RandomForestClassifier(criterion= 'entropy',
 max_depth= 8,
 n_estimators= 500)
scores = cross_val_score(rdf  , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8748224157977182

# Voting Classifier

In [30]:
from sklearn.ensemble import VotingClassifier

VC = VotingClassifier(estimators=[('xgb', XGBC), ('gb', GBC),('rf', rdf)], voting='soft', weights = [3,0.9,1.5])
scores = cross_val_score(VC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8766866969158326

In [32]:
#Modificacion de xgboost
from sklearn.ensemble import VotingClassifier

VC = VotingClassifier(estimators=[('xgb', XGB2), ('gb', GBC),('rf', rdf)], voting='soft', weights = [2.5,0.9,1.5])
scores = cross_val_score(VC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8770445827483527

In [34]:

VC.fit(df_X, df_y)
df_p = dfTest.merge(dfUsers, on='person').drop(['person','label'], axis=1)
temp = VC.predict_proba(df_p)
submit = []
for l in temp:
    submit.append(l[1])
dfTest['label'] = submit

KeyError: "labels ['label'] not contained in axis"

In [ ]:
dfTest.to_csv('trocafone_kaggle_submitB7.csv', index=False)

In [ ]:
pd.read_csv('trocafone_kaggle_submitB7.csv')